
<img src="http://hydrology.usu.edu/images/taudem5logo.gif" style="float:left;width:300px;padding:20px">       


# TauDEM Raster Processing Notebook

<br>
This notebook demonstrates basic watershed processing on using the TauDEM library.  For more information about TauDEM, see http://hydrology.usu.edu/taudem/taudem5/


## Script Setup and Preparation

In [ ]:
# import required libaries for geoprocessing
%matplotlib inline
import os
import shutil
import hs_utils
import taudem

# create an instance of the hydroshare rest interface
hs = hs_utils.hydroshare()

### Retrieve a raster resource using its ID

This example uses a digital elevation model (DEM) that is stored in HydroShare at the following url: https://www.hydroshare.org/resource/6ee06abb2ed8414f9b66d229c8e9a129/ .  The data for our processing routines can be retrieved by using the `getResourceFromHydroShare` function by passing in the global identifier from the url above.

In [ ]:
# get some resource content. The resource content is returned as a dictionary
content = hs.getResourceFromHydroShare('6ee06abb2ed8414f9b66d229c8e9a129')

Two file were downloaded onto the Python notebook server: (1) logan.tiff, and (2) logan.vrt.  We will be using the logan.tiff file as the starting point for our GIS processing.


### Prepare a directory for output files

The `taudem` library consists of utility functions for taudem GIS processing.  The `create_workspace` function will build a clean directory for our GIS processing results.

In [ ]:
# prepare a clean workspace for TauDEM output files
data_directory = taudem.create_workspace('logan_data_processing')   

---


##  Raster Processing

The following operations demonstrate how TauDEM can be used to perform basic raster processes. See the following link for more information on TauDEM:  
http://hydrology.usu.edu/taudem/taudem5/TauDEM53CommandLineGuide.pdf

### 1. View the Raw DEM

In [ ]:
# display the raw dem
raw_dem_path = hs.content['logan.tif']
taudem.plot_tiff(raw_dem_path)

### 2. Remove DEM Pits

In [ ]:
# Fill the DEM Pits

# set the output paths
fill = os.path.join(data_directory, 'loganfel.tif')
cmd = 'pitremove -z %s -fel %s' % (raw_dem_path, fill)
taudem.run_cmd(cmd, processors=4)

### 3. Calculate Flow Directions

In [ ]:
#####################
# D8 Flow Direction #
#####################

# set the output paths 
fdr = os.path.join(data_directory, 'fdr.tif')  # flowdir
sd8 = os.path.join(data_directory, 'sd8.tif')  # slope

cmd = 'd8flowdir -fel %s -sd8 %s -p %s' % (fill, sd8, fdr)
taudem.run_cmd(cmd, processors=4)


In [ ]:
#######################
# DInf Flow direction #
#######################

# set the output paths 
ang = os.path.join(data_directory, 'loganang.tif')  # flow angle
slp = os.path.join(data_directory, 'loganslp.tif')  # flow slope

cmd = 'dinfflowdir -fel %s -ang %s -slp %s' % (fill, ang, slp)
taudem.run_cmd(cmd, processors=4)


#### Plot the Results

In [ ]:
taudem.plot_tiff(tiff=[fdr, ang], title=['D8 Flow Direction', 'DInf Flow Angle'])
taudem.plot_tiff(tiff=[sd8, slp], title=['D8 Slope', 'DInf Slope'])

### 4. Calculate Contributing Area

In [ ]:
########################
# D8 Contributing Area #
########################

# set the output paths
ad8 = os.path.join(data_directory, 'loganad8.tif')  # D8 contributing area

# build the taudem command
cmd = 'aread8 -p %s -ad8 %s' % (fdr, ad8)
# -p:   D8 flow direction output file   
# -ad8: D8 contributing area file (output)  

# run the command
taudem.run_cmd(cmd, processors=4)

In [ ]:
##########################
# DInf Contributing Area #
##########################

# set the output paths
sca = os.path.join(data_directory, 'logansca.tif')  # D-Infinity contributing area

# build the taudem command
cmd = 'areadinf -ang %s -sca %s' % (ang, sca)                         
# -ang: D-infinity flow direction input file.                                  
# -sca: D-infinity area output file.                                             

# run the command
taudem.run_cmd(cmd, processors=4)

---
## Save the results back into HydroShare

Using the HydroShare rest api, you can create a new resource in HydroShare from a content file.

In [ ]:
# define HydroShare required metadata
title = 'TauDEM Processing of Logan Watershed'
abstract = 'This a group of TauDEM processing results that were derived using the Logan River DEM.'
keywords = ['TauDEM', 'Logan River', 'Geoprocessing']

# set the resource type that will be created.
rtype = 'genericresource'

# create a list of files that will be added to the HydroShare resource.
files = [hs.content['logan.tif'],          # input DEM for Logan Watershed
         hs.content['taudem_logan.ipynb'], # this notebook
         fill, # pit fill dem
         fdr,  # D8 flow direction
         sd8,  # D8 slope
         ang,  # DInf flow angle
         slp,  # DInf slope
         ad8,  # D8 contributing area
         sca   # DInf contributing area
        ]  

# Set the Logan DEM resource as the "parent" 
# (i.e. the new resource will be "derived from" the "6ee06abb2ed8414f9b66d229c8e9a129 resource)
parent_resource = '6ee06abb2ed8414f9b66d229c8e9a129'


In [ ]:
# create a hydroshare resource containing these data
resource_id = hs.createHydroShareResource(abstract, 
                                          title, 
                                          derivedFromId=parent_resource,
                                          keywords=keywords, 
                                          resource_type=rtype, 
                                          content_files=files, 
                                          public=False)